## 1. Welcome to Berkeley
<p>It's 1973, and – as one of the top-ranked universities in the United States – Berkeley has attracted thousands of applicants to its graduate school. But how many made the cut?</p>
<p>We will start off by loading the <code>UCBAdmissions</code> dataset, which is included in base R. This shows the number of students – male and female – who were admitted or rejected from the six largest departments at Berkeley. Since the dataset takes the unwieldly form of a three-dimensional array, we will convert it to tidy format using the <code>tidy</code> function from the <code>tidytext</code> package. Then we will be ready to start doing some analysis.</p>
<p><img src="https://assets.datacamp.com/production/project_567/img/Berkeley.jpg" alt="Berkeley Campus" title="Berkeley Campus"></p>
<p><em>Source: <a href="https://www.flickr.com/photos/brainchildvn/">brainchildvn</a> on Flickr</em></p>

In [0]:
# Load UCBAdmissions dataset
# .... YOUR CODE FOR TASK 1 ....

# Print dataset to console
print(UCBAdmissions)

# Load broom package
# .... YOUR CODE FOR TASK 1 ....

# Convert UCBAdmissions to tidy format
ucb_tidy <- ....

# Print tidy dataset to console
print(ucb_tidy)

## 2. Acceptance rate for men and women
<p>The data is more readable now that it is in tidy format, but – since it is split by department and displays raw counts – it is difficult for us to infer any kind of gender bias. To do that, we need to aggregate over department and ask ourselves, overall, what <em>proportion</em> of men and women were accepted into Berkeley in 1973.</p>
<p>Here we make use of the <code>dplyr</code> package for all of our data manipulation tasks. We aggregate over department using the <code>group_by</code> function to get the total number of men and women who were accepted into or rejected from Berkeley that year, as well as the proportion accepted in each case. That will leave us in a better place to understand any accusations of gender bias.</p>

In [0]:
# Load the dplyr library
library(dplyr)

# Aggregate over department
ucb_tidy_aggregated <- ucb_tidy %>% 
  .... %>% 
  summarize(n = ....) %>% 
  ungroup() %>% 
  group_by(Gender) %>% 
  mutate(prop = ....) %>% 
  ....

# Print aggregated dataset
print(ucb_tidy_aggregated)

## 3. Visualizing the discrepancy
<p>From the previous task, we can see that <strong>44.5% of male applicants</strong> were accepted into Berkeley, as opposed to <strong>30.4% of female applicants</strong>. Now we can start to see the problem. Did Berkeley's graduate admissions department really discriminate against women that year?</p>
<p>Before we consider alternative explanations, let's visualize the discrepancy through a simple bar chart using <code>ggplot2</code>. This won't add much to our understanding of the problem right now, but it will act as a useful reference point later on.</p>
<p>For clarity, we will format acceptance rate as a percentage using the <code>percent</code> function from the <code>scales</code> package.</p>

In [0]:
# Load the ggplot2 and scales packages
library(ggplot2)
library(scales)

# Prepare the bar plot
gg_bar <- ucb_tidy_aggregated %>% 
    ggplot(aes(x = ...., y = ...., fill = ....)) +
    geom_col() +
    geom_text(aes(label = ....), vjust = ....) +
    labs(title = "Acceptance rate of male and female applicants",
         subtitle = "University of California, Berkeley (1973)",
         y = "Acceptance rate") +
    scale_y_continuous(labels = ...., limits = ....) +
    guides(fill = FALSE)

# Print the bar plot
print(gg_bar)

## 4. Acceptance rate by department
<p>The bar plot confirms what we already knew – a higher proportion of men were accepted than women. But what happens when we separate the graph out by department?</p>
<p>Now we can return to our <code>ucb_tidy</code> dataset. After calculating the proportion of acceptances/rejections, we will plot a separate chart for each department using the <code>facet_wrap()</code> function in <code>ggplot2</code>. This will give us an idea of how acceptance rates differ by department, as well as by gender.</p>

In [0]:
# Calculate acceptance/rejection rate
ucb_by_dept <- ucb_tidy %>% 
    group_by(....) %>% 
    mutate(prop = ....) %>% 
    filter(Admit == ....)

# Print the dataset
print(ucb_by_dept)

# Prepare the bar plot for each department
gg_bar_faceted <- ucb_by_dept %>% 
  ggplot(aes(Gender, prop, fill = Gender)) +
  geom_col() +
  geom_text(aes(label = percent(prop)), vjust = -1) +
  labs(title = "Acceptance rate of male and female applicants",
       subtitle = "University of California, Berkeley (1973)",
       y = "Acceptance rate") +
  scale_y_continuous(labels = scales::percent, limits = c(0, 1)) +
  facet_wrap(....) +
  guides(....)

# Print the bar plot for each department
print(gg_bar_faceted)

## 5. Alternative explanations
<p>Now that we have separated out our analysis by department, the interpretation has changed rather dramatically. Although men were indeed more likely to be admitted into Departments C and E, women were more likely to be admitted into all other departments. So what's really going on here?</p>
<p>If you turn your attention to the first two plots, you can see that Department A and B were quite easy to get into. However, relatively few women applied to these departments – only 108 women applied to Department A, as opposed to 825 men!</p>
<p>At this stage, we can hypothesise that the effect of gender on acceptance is null when you control for department. We can test that hypothesis using <strong>binary logistic regression</strong>, but first we need to de-aggregate the dataset so that each row represents one student. That should leave us with 4,526 rows – one row for each student who applied to Berkeley that year.</p>

In [0]:
# Define function that repeats each row in each column n times
multiply_rows <- function(....) {
  rep(....)
}

# Create new de-aggregated data frame using the multiply_rows function
ucb_full <- data.frame(Admit = ....(ucb_tidy$...., ucb_tidy$....),
                      Gender = ....(ucb_tidy$...., ucb_tidy$....),
                      Dept = ....(ucb_tidy$...., ucb_tidy$....))

# Check the number of rows equals the number of students
# .... YOUR CODE FOR TASK 5 ....

## 6. Binary logistic regression: part i
<p>The data is now in the right format for us to do some hypothesis testing. Great! But first let's try to predict admittance using gender alone. We will use the built-in <code>glm()</code> function to fit a generalised linear model, making sure to set <code>family = "binomial"</code> because the outcome variable is binary (<code>Admitted</code> or <code>Rejected</code>).</p>
<p>By default, <code>Admit</code> is coded such that <code>Admitted</code> is level 1 and <code>Rejected</code> is level 2 (because of their alphabetical order). Since <code>glm()</code> will assume that level 2 represents 'success', we will reverse the coding of <code>Admit</code> so we are predicting the probability of admittance rather than rejection.</p>
<p>To change the coding of a variable, you can use the <code>fct_relevel()</code> function from the <code>forcats</code> package.</p>

In [0]:
# Load the forcats library
library(forcats)

# Reverse the coding of the Admit variable
ucb_full$Admit <- ....

# Run the regression
glm_gender <- glm(Admit ~ ...., data = ucb_full, family = "....")

# Summarize the results
# .... YOUR CODE FOR TASK 6 ....

## 7. Binary logistic regression: part ii
<p>Sure enough, when you predict the probability of admission as a function of gender alone, the effect is statistically significant (p &lt; 0.01). Specifically, you are <code>exp(0.61035) = 1.84</code> times more likely to be admitted if you are a man. However, what happens if we control for department?</p>

In [0]:
# Run the regression, including Dept as an explanatory variable
glm_genderdept <- glm(Admit ~ ...., data = ucb_full, family = "....")

# Summarize the results
summary(....)

## 8. Behold Simpson's paradox
<p>Finally, we can see Simpson's paradox at play – when you control for the effect of department on the probability of admission, the effect of gender disappears. In fact, it even reverses, suggesting that – controlling for department – you were actually more likely to be admitted as a woman! However, this effect is not statistically significant (p &gt; 0.05), so we conclude that there was not a campus-wide bias against applicants of either gender in 1973.</p>
<p>That said, individual departments often handle their own admissions processes, so it is plausible that bias exists in one department but not another. Let's take a look at Department A, where 82.4% of women were admitted but only 62.1% of men. Is the difference statistically significant?</p>

In [0]:
# Filter for Department A
dept_a <- ....

# Run the regression
glm_gender_depta <- ....

# Summarize the results
summary(....)

## 9. Bias or discrimination?
<p>Well then! If we take Department A in isolation, we find there is a statistically significant bias in favour of women. So does that mean that the department discriminated against men?</p>
<p>Not necessarily. After all, the bias might exist simply because the female applicants to Department A were better qualified that year. In their article dealing with this issue, Bickel, Hammel &amp; O'Connell (1975) define discrimination as "the exercise of decision influenced by the sex of the applicant when that is immaterial to the qualifications for entry". Since we do not have any data on the respective qualifications of the candidates, we cannot say whether any gender bias in their admissions process amounted to discrimination.</p>
<p>Although now more than 40 years old, the Berkeley problem is a useful reminder about the dangers of aggregation and omitted variable bias, especially in relation to matters of such legal and ethical importance as discrimination. Where bias does exist – as it does in the case of Department A – it is always worth considering whether there are any other factors that could explain the discrepancy.</p>

In [0]:
# Define bias
bias <- "...."

# Define discrimination
discrimination <- "...."

# Is bias equal to discrimination?
# .... YOUR CODE FOR TASK 9 ....